# pgpy

pgpy is a versatile python library designed for variants analysis contained in VCF files.

## About

#### Why this lib

pgpy does not intend to be a fast VCF processing tool but much more to allow you to quickly make specific analysis on your variants. In this context (see peformance), several other tools should be (way) faster compared to this module for some analysis. However, this module allows you to have a complete controle of your data and to easily produce scripts for very specific purposes.

#### Formatting your data 

While this module allows you to modify your data on the fly, it is important to correctly manage your data before. This module allows you to read both single sample VCF or concatened VCFs produced either by [GATK GenotypeGVCFs](https://software.broadinstitute.org/gatk/documentation/tooldocs/3.8-0/org_broadinstitute_gatk_tools_walkers_variantutils_GenotypeGVCFs.php) or [VCFTools merge](https://vcftools.github.io/perl_module.html#vcf-merge) (use GATK is you can). VCF files must be tabixed and it is not a problem to bgzip them too, everything is managed by pysam for this. It is highly recommanded to set a proper ploidy to your sample (which can be done with GATK) for example.

#### Requirements and compatibility

This module should work if you use python 3 version. However, it has only be tested with my configuration, which is :

- Python 3.5.4
- Pysam 0.11.2.2
- BioPython 1.68

#### Performance issues

Since this module is written in pure python, it can be slow especialy if your analysis is based on whole genome data. This is mainly due to the poor loop performance in python which are highly used in this context (iteration through sites and variants). On way to reduce this is to compiled your code using cython for example, however it is really unlikely that you will compete with other tools such as VCFTools.

## Basics

To iterate through your site, a `VCFIterator` object can be used. Since pysam manages tabix file, it allows you to select either whole genome or specific genome regions. Basic informations can also be accessed easily.

In [2]:
from pgpy import VCFIterator as VI

vcf = VI("./Test/vcf.gz")
print (vcf.samples[:5])
print (vcf.contigs)

['AAA', 'AAB', 'AAC', 'AAD', 'AAE']
['KLLA0A', 'KLLA0B', 'KLLA0C', 'KLLA0D', 'KLLA0E', 'KLLA0F']


In [3]:
for site in vcf.fetch() :
    contig, position, ref, variants = site
    print (contig, position, ref)
    print (variants)
    break

KLLA0A 13297 C
{'AAW': ('C',), 'AAV': ('C',), 'ABB': ('C',), 'AAA': ('C',), 'ABS': ('C',), 'ABI': ('C',), 'AAC': ('C',), 'ABA': ('C',), 'ABM': ('C',), 'AAS': ('C',), 'ABK': ('C',), 'AAU': ('C',), 'AAF': ('C',), 'AAQ': ('C',), 'ABG': ('C',), 'AAX': ('C',), 'AAJ': ('T',), 'ABE': ('C',), 'AAO': ('C',), 'AAN': ('C',), 'ABF': ('T',), 'AAL': ('C',), 'AAR': ('C',), 'ABP': ('C',), 'ABR': ('C',), 'AAM': ('C',), 'ABQ': ('C',), 'AAP': ('C',), 'AAE': ('T',), 'ABD': ('C',), 'AAK': ('C',), 'AAY': ('C',), 'AAT': ('C',), 'AAG': ('C',), 'AAI': ('T',), 'ABC': ('C',), 'AAD': ('C',), 'AAH': ('T',), 'ABO': ('C',), 'AAZ': ('C',), 'AAB': ('C',)}


For the rest of the document, I will use the function `fetch_first` which return just the first site.

#### Modifiers

As you can see, pysam gives all alleles for each sample and in this case only haploid sample can be found. One of the strength of this lib is to be able to change on the fly alleles information. 

To do this, modifiers function can be used to modify allele during parsing. Some modifier are alread proposed in this lib, such as :

- iupac_modifier
- no_indel_modifier
- only_indel_modifier
- only_variable_modifier

In [5]:
vcf.modifier = VI.only_variable_modifier
print (vcf.fetch_first())

('KLLA0A', 13297, 'C', {'AAI': ('T',), 'AAJ': ('T',), 'ABF': ('T',), 'AAE': ('T',), 'AAH': ('T',)})


However, you can also build specific modifier depending of what you want

In [7]:
def custom_modifier(variants, site) :
    variants = VI.only_variable_modifier(variants, site)
    return variants[0] if variants else variants

vcf.modifier = custom_modifier
print (vcf.fetch_first())

('KLLA0A', 13297, 'C', {'AAI': 'T', 'AAJ': 'T', 'ABF': 'T', 'AAE': 'T', 'AAH': 'T'})


#### Build-in site functions

I try to add some usefull function using variants, for example if you need allele frequency

In [9]:
vcf.modifier = None
for site in vcf.fetch() :
    contig, position, ref, variants = site
    print (VI.acount(variants, freq=True))
    break

{('C',): 0.8780487804878049, ('T',): 0.12195121951219512}


## Alignments

Producing output files, including alignment with reference data is as easy 

In [11]:
from pgpy.core.aln import snps_aln

ref = "./Test/ref.fasta"

# be sure that we dont have wrong modifier on the way
vcf.modifier = None

aln = snps_aln(vcf, ref, contig="KLLA0A", start=0, stop=100)
print (aln)

Alphabet() alignment with 41 rows and 100 columns
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAA_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAB_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAC_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAD_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAE_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAF_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAG_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAH_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAI_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAJ_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAK_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAL_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAM_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAN_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAO_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAP_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAA

This function returns a [`MultipleSeqAlignment`](https://biopython.org/DIST/docs/api/Bio.Align.MultipleSeqAlignment-class.html) (MSA) object which can be exported easily in other format using [biopython AlignIO](https://biopython.org/wiki/AlignIO).

In [12]:
from Bio import AlignIO

outfile = "./Test/aln.phylip"
with open(outfile, "w") as f :
    AlignIO.write(aln, f, "phylip")

Note that MSA objects can be summed, allowing you to easily concat alignments (such as contigs for whole genome).

In [13]:
aln2 = aln + aln
print (aln2)

Alphabet() alignment with 41 rows and 200 columns
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAA_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAB_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAC_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAD_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAE_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAF_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAG_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAH_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAI_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAJ_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAK_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAL_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAM_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAN_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAO_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAAATCCGT...CCA AAP_1
AACCGGCGCCAGTGTGCTGGGACCACATACCTAATCAA

## Recipes

I try to add some function which could be reused easily on other data. They can be found in the recipes directory. For example for minor allele frequency (require pandas) :

In [15]:
from pgpy import *

res = maf("./Test/vcf.gz", "KLLA0A", 0, 100000)
print (res)

    SampleCount  SampleFreq  SiteCount  SiteFreq
0           9.0    0.073171       2975  0.481080
1          12.0    0.097561         44  0.007115
2          15.0    0.121951        242  0.039133
3          18.0    0.146341        999  0.161546
4          21.0    0.170732         17  0.002749
5          24.0    0.195122        108  0.017464
6          27.0    0.219512        395  0.063875
7          30.0    0.243902        544  0.087969
8          33.0    0.268293         40  0.006468
9          36.0    0.292683        108  0.017464
10         39.0    0.317073         33  0.005336
11         42.0    0.341463          2  0.000323
12         45.0    0.365854         22  0.003558
13         48.0    0.390244         16  0.002587
14         51.0    0.414634          3  0.000485
15         54.0    0.439024         50  0.008085
16         57.0    0.463415         97  0.015686
17         60.0    0.487805        489  0.079075


## Multiprocessing

While pgpy is not made for performance, a multiprocessing tool dedicated to query different regions at the same time is available. This function requiered that the functions arguments take `contig`, `start` and `end` arguments, either as positional argument or as `** kwargs`. Most of the recipies allows this, however concatenation of the result requiered often a bit of knowledge in pandas.

Two functions are available : 
- The first one (`apply_mp`) gives you a complete controle of your regions
- The second one (`apply_mp_contig`) is simplier and does the trick for each contig of your vcf

For the sake of this tutorial, let's say we only want to count the number of SNPs in one of our haploid samples. First let make this without multi-processing.

In [31]:
VI.SHOWMODE = True
VI.SHOWPOSI = 500000 # show the loop position each 500kb

def count_indels(vcf, sample, ** kwargs) :
    count = 0

    for site in vcf.fetch(** kwargs) :
        ref = site[2]
        genotype = site[3]
        
        # This work because our sample is haploid
        if genotype[sample][0] != None :
            count += 1
        
    return count
    
vcf = VI("./Test/vcf.gz")
print (count_indels(vcf, "AAA"))

KLLA0A 13297
KLLA0A 500004
KLLA0A 1000001
KLLA0B 18321
KLLA0B 500022
KLLA0B 1000001
KLLA0C 14806
KLLA0C 500011
KLLA0C 1000006
KLLA0C 1500051
KLLA0D 18315
KLLA0D 500027
KLLA0D 1000011
KLLA0D 1514015
KLLA0E 22584
KLLA0E 500001
KLLA0E 1000030
KLLA0E 1500029
KLLA0E 2000192
KLLA0F 12969
KLLA0F 500025
KLLA0F 1000085
KLLA0F 1500037
KLLA0F 2000013
KLLA0F 2500004
756589


Now we can make it easily scale up for each contig

In [36]:
from pgpy.core.processing import apply_mp_contig

results = apply_mp_contig(vcf, count_indels, "AAA", ncore=4)
print (results)

KLLA0D 18315
KLLA0A 13297
KLLA0C 14806
KLLA0B 18321
KLLA0D 500027
KLLA0C 500011
KLLA0A 500004
KLLA0B 500022
KLLA0D 1000011
KLLA0B 1000001
KLLA0A 1000001
KLLA0C 1000006
KLLA0E 22584
KLLA0F 12969
KLLA0D 1514015
KLLA0C 1500051
KLLA0E 500001
KLLA0F 500025
KLLA0E 1000030
KLLA0F 1000085
KLLA0E 1500029
KLLA0F 1500037
KLLA0E 2000192
KLLA0F 2000013
KLLA0F 2500004
{'KLLA0E': 158884, 'KLLA0B': 92410, 'KLLA0A': 72796, 'KLLA0F': 189865, 'KLLA0D': 117931, 'KLLA0C': 124703}


As you can see, 4 contigs has been processed at the same time, however each result are splitted by contigs.

In [37]:
print (sum(results.values()))

756589


With this example, you might not have seen a big difference in the processing time. This is due to the fact that most of the processing time was in fact link to the file reading. Since we iterate through the same file, it's not that faster to use different process. However, it becomes really handy when you use more "heavy" function.